In [22]:
import pandas as pd
import data_util
from tqdm.notebook import tqdm
#from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
import numpy as np
from data_util import *
from sklearn_model_hyper import *

import threading
import multiprocessing

import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scs
import scikitplot as skplt


from tensorflow.keras.layers import InputLayer, BatchNormalization, GlobalMaxPool1D, Bidirectional, Dense, Flatten, Conv2D, LeakyReLU, Dropout, LSTM, GRU, Input
from tensorflow.keras import Model, Sequential
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import regularizers
from keras.wrappers.scikit_learn import KerasClassifier
from imblearn.over_sampling import RandomOverSampler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import tensorflow.keras as keras
import random

from backtest import *


from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score


In [23]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
import numpy as np



In [24]:
path = "./data/"
trainX_raw, trainY_raw = load_data("", "train", path)
valX_raw, valY_raw = load_data("", "Val", path)

#trainX_balanced, trainY_balanced = get_balanced_set(trainX_raw, trainY_raw)

valX, valY = valX_raw, valY_raw

features = trainX_raw.shape[-1]


FileNotFoundError: [Errno 2] No such file or directory: './data/trainX.npy'

In [25]:
#print(Counter(trainY))


X_over, y_over = get_balanced_set(trainX_raw, trainY_raw)

print()
print("{} {}".format(Counter(y_over), Counter(trainY_raw)))

normalizer = tf.keras.layers.experimental.preprocessing.Normalization(input_shape=(features,))
normalizer.adapt(trainX_raw)

print("{} {} | {} {}".format(X_over.shape, y_over.shape, valX.shape, valY.shape))


ValueError: Unknown label type: 'continuous'

In [26]:
step = 5

trainX_raw, trainY_raw = load_data("simple_full_", "train", path)
x, y, closed_prices = prepare_train_data(trainX_raw, trainY_raw, step)

X_over, y_over = get_balanced_set(x, y)


normalizer = tf.keras.layers.experimental.preprocessing.Normalization(input_shape=(features,))
normalizer.adapt(trainX_raw)

trainX_raw, trainY_raw = load_data(f"simple_backtest", "train", path)
valX, valY, closed_prices = get_sequencial_data(trainX_raw, trainY_raw, step)

In [28]:

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

def set_seeds(seed=100):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(100)


metrics = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

#metric = 'accuracy'

In [29]:
%%time

def get_full_model(
                  features = 3, 
                  layers = 1, 
                  units = 5, 
                  optimizer='Adam',
                  activation='relu',
                  k_reg=regularizers.l2(0.01),
                  init='glorot_uniform',
                  lays_seqs = 3
                 ):

    set_seeds()
    
    #print("{} {}".format(activation, optimizer))

    model = Sequential()

    #, input_shape=(seqs, features)
    
    model.add(normalizer)
        
    model.add(Dense(units, 
                  kernel_initializer=init,
                  kernel_regularizer=k_reg,
                  activation=activation))
    

    for lay in range(int(layers)):
        model.add(Dense(units, 
                        kernel_initializer=init, 
                        kernel_regularizer=k_reg,
                        activation=activation))

    model.add(Dropout(0.5))

    model.add(Dense(1, 
                  kernel_initializer=init,
                  kernel_regularizer=k_reg,
                  activation='sigmoid')) 

    model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=metrics)
    
    return model


#model = get_rnn_model(**params)
#model.summary()

CPU times: user 51 µs, sys: 56 µs, total: 107 µs
Wall time: 114 µs


In [30]:
%%time
# Get_full_model


ea = tf.keras.callbacks.EarlyStopping(
    monitor="val_precision",
    min_delta=0,
    patience=10,
    verbose=1,
    mode="auto",
    restore_best_weights=True
)

model = KerasClassifier(build_fn=get_full_model, 
                        callbacks=[ea],
                        validation_data = (valX, valY),
                        verbose=0
                       )
# grid search epochs, batch size and optimizer


init = [
        'glorot_uniform', 
        #, 'normal' 
        'he_uniform'
        ]


X_train, Y_train = X_over, y_over


param_grid = dict(
    init = init,
    optimizer = [
              "Adadelta",
              "Nadam"
              , "Adam"
            ], 
    epochs=[100], 
    batch_size=[64],
    layers = [
        5, 
        15, 
        20],
    units = [
        16,
        32,
        64
            ],
    features = [features],
    k_reg=[ 
        "l2", 
        "l1"
    ],
    activation = [
        #'softmax', 
        #'softplus', 
        #'softsign', 
        'tanh', 
        'relu', 
        'sigmoid', 
        #'hard_sigmoid', 
        #'linear'
        ]
    )



CPU times: user 354 µs, sys: 316 µs, total: 670 µs
Wall time: 678 µs


In [ ]:
%%time


clf_grid = gridSearch(X_over, y_over, model, param_grid)

#10181818181818181

Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping


/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping


/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Restoring model weights from the end of the best epoch.
Epoch 00014: early stopping


...

/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping


/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping


/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
clf_grid.best_params_

param_test = clf_grid.best_params_

del param_test['batch_size']
del param_test['epochs']

param_test

In [ ]:
%%time


#model = get_full_model(**clf_grid.best_params_)
#model = get_full_model(**param_test)

#x_norm = normalizer(X_over).numpy()

#model.fit(
#          X_over, y_over,
#          epochs=100, 
#          verbose=True,
#          batch_size=64,
#          callbacks=[ea],
#          validation_data = (valX, valY),
#          )

#model.summary()

In [ ]:
model = clf_grid.best_estimator_

In [ ]:
valX_raw, valY_raw = load_data("backtest", "train", path)


val_x_norm = normalizer(valX).numpy()

eval_data(model, val_x_norm, valY)

eval_data(model, valX_raw, valY_raw)
